In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import datetime as dt
import time

After imports, read in the dataset<br>(note that we have both train_set AND test_set now... should think about including the test_set in our training data, so it has more results to train on?)

In [2]:
# read in the datasets
train_data_path = '../csv_files/booking_train_set.csv'
test_data_path = '../csv_files/booking_test_set.csv'

# import train_data set into pandas frame
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

### adding necessary columns to the datasets ###

In [3]:
# cast dates to datetime format
for df in [train_data, test_data]:
    df['checkin'] = pd.to_datetime(df['checkin'])
    df['checkout'] = pd.to_datetime(df['checkout'])

In [4]:
# create a column that holds the length of each 'leg' of the trip
for df in [train_data, test_data]:
    df['length_of_leg'] = (df['checkout'] - df['checkin']).dt.days

### stripping out target variable Y from train_data and creating training data w/o target variable ###

In [5]:
# series that holds a list of cities visited on each trip
orderofcitiesvisitedpertrip = train_data.groupby('utrip_id')['city_id'].agg(lambda x: list(x))

In [6]:
# pull out the last city of each utrip_id into a series
lastcityoftripseries = orderofcitiesvisitedpertrip.apply(lambda x: x[-1])
utrip_ids = lastcityoftripseries.index.tolist()
lastcityoftrip = lastcityoftripseries.tolist()

In [7]:
# store our target in train_Y which tells us our target variable for each trip 
Y_train = list(zip(utrip_ids, lastcityoftrip))

In [8]:
# store our training data in train_X which will holds all of the trips from each trip except for the last stop
X_train = train_data.drop(train_data.groupby('utrip_id').tail(1).index, axis=0)

#### Goals for tonight's meeting ####
<ul>
    <li> drop unneccessary columns from X_train</li>
    <li> one-hot-encode the following columns:</li>
    <li> week_of_year (1 of 52)</li>
    <li> days_of_week (1 of 7)</li>
    <li> weekday_weekend (this might be more helpful as a metric rather than days of the week)</li>
    <li> device_class (1 of 3)</li>
    <li> booker_country (1 of 5)</li>
    <li> hotel_country (1 of 195)</li>
    <li> city_id (1 of 39901)</li>
    <li> length_of_leg (normalized)</li>
</ul>

In [9]:
X_train.head(10)

,Unnamed: 0,user_id,checkin,checkout,city_id,device_class,affiliate_id,booker_country,hotel_country,utrip_id,length_of_leg
0,0,1006220,2016-04-09,2016-04-11,31114,desktop,384,Gondal,Gondal,1006220_1,2
1,1,1006220,2016-04-11,2016-04-12,39641,desktop,384,Gondal,Gondal,1006220_1,1
2,2,1006220,2016-04-12,2016-04-16,20232,desktop,384,Gondal,Glubbdubdrib,1006220_1,4
4,4,1010293,2016-07-09,2016-07-10,5325,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1
5,5,1010293,2016-07-10,2016-07-11,55,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1
6,6,1010293,2016-07-12,2016-07-13,23921,mobile,359,The Devilfire Empire,Cobra Island,1010293_1,1
7,7,1010293,2016-07-13,2016-07-15,65322,desktop,9924,The Devilfire Empire,Cobra Island,1010293_1,2
8,8,1010293,2016-07-15,2016-07-16,23921,desktop,9924,The Devilfire Empire,Cobra Island,1010293_1,1
10,10,1012680,2016-10-23,2016-10-25,37709,desktop,384,Gondal,Yerba,1012680_1,2
11,11,1012680,2016-10-25,2016-10-27,11837,desktop,384,Gondal,Panem,1012680_1,2


In [10]:
# get the week the stay began
X_train['checkin'].dt.isocalendar().week

0          14
1          15
2          15
4          27
5          27
           ..
1166828    36
1166829    36
1166831    50
1166832    50
1166833    51
Name: week, Length: 949149, dtype: UInt32